# ACTA 6. Selección de muestras

En esta acta el objetivo es realizar la selección de la muestra para aplicar los modelos estadísticos que vayamos a realizar. 

Los modelos estadísticos a emplear serán:

- Clustering: a partir de las variables categóricas codificadas aplicaremos un clustering con el objetivo de explorar nuestro conjunto e datos, su comprotamiento y la existencia de tendencias o patrones para diferentes conjuntos de individuos. La aplicación del aprendizaje no supervisado en esta etapa tiene como objetivo extraer conclusiones que nos permitar tomar decisiones en la aplicación del segundo modelo estadístico. 

- Redes neuronales Transformer: a partir de las variables categóricas y numéricas normalizadas predeciremos la fecha, latitud y longitud.  

Para ambos modelos estadísticos lo que realizaremos para reducir la complejidad por falta de tiempo y recursos computacionales seleccionaremos los años 2021,2022 y 2023 y seleccionaremos una muestra estratificada. 

La elección de la muestra estratificada se basará en mantener las proporciones de las variables originales de esta forma evitaremos que nuestros resultados se vean afectados por la selección de la muestras. 

A continuación seguiremos los siguientes pasos para realizar lo explicado anteriormente:

1. Carga del conjunto de datos y visualización de las dimensiones y columnas

2. Selección de los años 2021, 2022 y 2023. Dimensiones

3. Selección de la muestra estratificada 

In [ ]:
# 1. Carga del conjunto de datos
import pandas as pd

Incidents = pd.read_csv('Incidents.csv')
Incidents.head()

,Unnamed: 0,incident_datetime,incident_day_of_week,incident_code,incident_category,incident_subcategory,resolution,latitude,longitude,neighborhood,...,hour,minutes,incident_day_of_week_cod,incident_category_cod,incident_subcategory_cod,resolution_cod,neighborhood_cod,Police_district_cod,Holiday_cod,Street_cod
0,0,2020-06-22 08:00:00,Monday,9029,FRAUD,FRAUD,Open or Active,37.772618,-122.483119,Outer Richmond,...,8,0,0,0,0,0,0,0,0,0
1,1,2020-06-22 13:30:00,Monday,7043,RECOVERED VEHICLE,RECOVERED VEHICLE,Open or Active,37.765117,-122.418579,Mission,...,13,30,0,1,1,0,1,1,0,1
2,2,2020-06-22 22:26:00,Monday,12026,ASSAULT,AGGRAVATED ASSAULT,Open or Active,37.728942,-122.378542,Bayview Hunters Point,...,22,26,0,2,2,0,2,2,0,2
3,3,2020-06-22 18:00:00,Monday,6243,LARCENY THEFT,LARCENY - FROM VEHICLE,Open or Active,37.712320,-122.456045,Oceanview/Merced/Ingleside,...,18,0,0,3,3,0,3,3,0,3
4,4,2020-06-22 11:30:00,Monday,6244,LARCENY THEFT,LARCENY - FROM VEHICLE,Open or Active,37.806658,-122.420560,Russian Hill,...,11,30,0,3,3,0,4,4,0,4


In [ ]:
Incidents.drop(columns = ['Unnamed: 0'], inplace = True)
Incidents.columns

Index(['incident_datetime', 'incident_day_of_week', 'incident_code',
       'incident_category', 'incident_subcategory', 'resolution', 'latitude',
       'longitude', 'neighborhood', 'Police_district', 'Holiday', 'Street',
       'day', 'month', 'year', 'hour', 'minutes', 'incident_day_of_week_cod',
       'incident_category_cod', 'incident_subcategory_cod', 'resolution_cod',
       'neighborhood_cod', 'Police_district_cod', 'Holiday_cod', 'Street_cod'],
      dtype='object')

In [ ]:
# 2. Selección de los años 2021,2022,2023
Incidents_years = Incidents[Incidents['year'].isin(range(2021,2024))]
Incidents_years['year'].value_counts()

year
2022    126222
2021    121958
2023     32692
Name: count, dtype: int64

In [ ]:
Incidents_years.shape

(280872, 25)

In [ ]:
import numpy as np
from joblib import Parallel, delayed


data = Incidents_years.copy()
sample_size = 30000


variable_proportions = data.apply(lambda x: x.value_counts(normalize=False))


new_proportions = variable_proportions * sample_size


def stratified_sampling(data, variable, proportions, size, min_samples=2):
    valid_classes = proportions[proportions >= min_samples].index
    sample = data[data[variable].isin(valid_classes)]
    sample = sample.groupby(variable, group_keys=False).apply(lambda x: x.sample(n=int(np.round(proportions[x.name])), replace=True))
    return sample


sample_train = data.copy()

def stratified_parallel(variable):
    return stratified_sampling(sample_train, variable, new_proportions[variable], sample_size)

with Parallel(n_jobs=-1) as parallel:
    sample_train = parallel(delayed(stratified_parallel)(variable) for variable in new_proportions)


sample_train = pd.concat(sample_train)
sample_train = sample_train.sample(frac=1, random_state=1)


print("Proporciones en el conjunto de muestra:\n", sample_train.apply(lambda x: x.value_counts(normalize=True)))
